In [29]:
import numpy as np
import cv2 as cv
import os

In [24]:
def rescaleFrame(frame, scale=0.75):
    width = int(frame.shape[1]*scale)
    height = int(frame.shape[0]*scale)
    
    dimensions = (width, height)
    
    return cv.resize(frame, dimensions, interpolation=cv.INTER_AREA)

In [30]:
img_raw = cv.imread('Raw Data/chess_board_one.png')

resized_img = rescaleFrame(img_raw)
cv.imshow('Lichess', resized_img)

cv.waitKey(0)

-1

In [26]:
def find_chessboard_corners(img):
    img_with_contour = img.copy()
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    blur = cv.GaussianBlur(gray, (5, 5), 0)
    edges = cv.Canny(blur, 50, 150)
    contours, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # if len(contours) > 0:
    largest_contour = max(contours, key=cv.contourArea)
    
        
    points = largest_contour.reshape(len(largest_contour), 2)

    # Create a new array to store the 4 corner points
    rect = np.zeros((4, 2), dtype="float32")

    # The top-left point has the smallest sum, bottom-right has the largest
    s = points.sum(axis=1)
    rect[0] = points[np.argmin(s)] # Top-left
    rect[2] = points[np.argmax(s)] # Bottom-right

    # The top-right point has the largest difference, bottom-left has the smallest
    diff = np.diff(points, axis=1)
    rect[1] = points[np.argmin(diff)] # Top-right
    rect[3] = points[np.argmax(diff)] # Bottom-left
    
    rect_int = np.int32(rect)
    cv.drawContours(img_with_contour, [rect_int], -1, (0,255,3), 1)
    
    
            # Display the result
    cv.imshow('Original Image', img)
    cv.imshow('Edges', edges)
    cv.imshow('Chessboard Contour', img_with_contour)
    cv.waitKey(0)
    cv.destroyAllWindows()
    return rect
find_chessboard_corners(resized_img)

array([[442.,  62.],
       [956.,  64.],
       [953., 577.],
       [441., 574.]], dtype=float32)

In [27]:
def create_dataset_from_corners(original_image, rect, output_dir='dataset'):
    """
    Warps the perspective of a chessboard using provided corners, crops each square, 
    and saves them into labeled folders.
    """
    # Define the destination points for our 800x800 warped image
    dst_points = np.array([
        [0, 0], [799, 0], [799, 799], [0, 799]], dtype="float32")

    # Get the perspective transform matrix and apply it
    M = cv.getPerspectiveTransform(rect, dst_points)
    warped_board = cv.warpPerspective(original_image, M, (800, 800))
    print("Board perspective transformed. Cropping and saving squares...")

    # Hardcoded map of the starting position
    piece_map = [
        ['br', 'bn', 'bb', 'bq', 'bk', 'bb', 'bn', 'br'],
        ['bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp'],
        ['e', 'e', 'e', 'e', 'e', 'e', 'e', 'e'], ['e', 'e', 'e', 'e', 'e', 'e', 'e', 'e'],
        ['e', 'e', 'e', 'e', 'e', 'e', 'e', 'e'], ['e', 'e', 'e', 'e', 'e', 'e', 'e', 'e'],
        ['wp', 'wp', 'wp', 'wp', 'wp', 'wp', 'wp', 'wp'],
        ['wr', 'wn', 'wb', 'wq', 'wk', 'wb', 'wn', 'wr']
    ]

    square_size = 100
    img_counts = {}

    for row in range(8):
        for col in range(8):
            y1, y2 = row * square_size, (row + 1) * square_size
            x1, x2 = col * square_size, (col + 1) * square_size
            square_img = warped_board[y1:y2, x1:x2]
            piece_label = piece_map[row][col]
            
            piece_dir = os.path.join(output_dir, piece_label)
            os.makedirs(piece_dir, exist_ok=True)
            
            count = img_counts.get(piece_label, 0)
            filename = f"{piece_label}_{count}.png"
            filepath = os.path.join(piece_dir, filename)
            
            cv.imwrite(filepath, square_img)
            img_counts[piece_label] = count + 1

    print(f"Dataset successfully created in the '{output_dir}' directory.")
    cv.imshow("Warped Chessboard for Verification", warped_board)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [28]:
# --- Main Script ---
if __name__ == '__main__':
    
    # Assuming you resize it
    # resized_img = cv.resize(original_img, (new_width, new_height))
    # For now, let's just use the original

    # Step 1: Get the corner coordinates from your function
    board_corners = find_chessboard_corners(resized_img)
    
    # Step 2: If corners were found, pass them to the dataset creation function
    if board_corners is not None:
        print("Corners found. Proceeding to create dataset.")
        create_dataset_from_corners(resized_img, board_corners)
    else:
        print("Could not find chessboard corners.")

Corners found. Proceeding to create dataset.
Board perspective transformed. Cropping and saving squares...
Dataset successfully created in the 'dataset' directory.
